In [ ]:
!pip install dataprep

In [ ]:
! pip  install pyspark

In [ ]:
import warnings
import pandas as pd 
from dataprep.eda import create_report
import pyspark


In [ ]:
url='https://drive.google.com/file/d/1TgndvrlNhBShoJy74hXLtFXZ6HGWDo8w/view?usp=sharing'
url='https://drive.google.com/uc?id=' + url.split('/')[-2]
df = pd.read_csv(url)
df.to_csv("/content/Course_info.csv",index=False)


In [ ]:
df = pd.read_csv("/content/Course_info.csv")

In [ ]:
df.info()

In [ ]:
df['category'].unique()

In [ ]:
df['topic'].nunique()

In [ ]:
df['subcategory'].nunique()

In [ ]:
create_report(df)

In [ ]:
spark = pyspark.sql.SparkSession.builder.getOrCreate()

In [ ]:
from pyspark.sql.types import StructType,StructField, StringType, IntegerType, BooleanType,FloatType,TimestampType


In [ ]:
schema = StructType([ \
    StructField("id",IntegerType(),True), \
    StructField("title",StringType(),True), \
    StructField("is_paid",BooleanType(),True), \
    StructField("price", FloatType(), True), \
    StructField("headline", StringType(), True), \
    StructField("num_subscribers", FloatType(), True), \
    StructField("avg_rating", FloatType(), True), \
    StructField("num_reviews", FloatType(), True), \
    StructField("num_comments", FloatType(), True), \
    StructField("num_lectures", FloatType(), True), \
    StructField("content_length_min", FloatType(), True), \
    StructField("published_time", TimestampType(), True), \
    StructField("last_update_date", TimestampType(), True), \
    StructField("category", StringType(), True), \
    StructField("subcategory", StringType(), True), \
    StructField("topic", StringType(), True), \
    StructField("language", StringType(), True), \
    StructField("course_url", StringType(), True), \
    StructField("instructor_name", StringType(), True), \
    StructField("instructor_url", StringType(), True), \
  ])

In [ ]:
data = spark.read.csv("/content/Course_info.csv",
                     sep=',',
                     schema=schema,
                     header=True,
                     multiLine=True)

data.printSchema()

In [ ]:
print((data.count(), len(data.columns)))

In [ ]:
data.registerTempTable('data')

Questions:
1. Who are the most popular instructors?
2. What are the most popular language?
3. What are the most popular category?
4. What are the most popular subcategory?
5. What are the most popular topic?
6. what are the total of paid and free courses ?
 
---

7. What are the best courses according to the number of subscribers?
8. What are the best category according to the number of subscribers?
9. What are the total courses for each instructor ?
10. What are the best predictors of course popularity?
11. What are the most popular topics for each category?
12. Who are the most popular instructorsin each category?
 

---



# Single Feature

The most frequent of :
1. language
2. instructor
3. category
4. subcategory
5. is_paid
6. topic


In [ ]:
state_counts = spark.sql(r"""SELECT language, COUNT(language) AS Count 
                                     FROM data 
                                     GROUP BY language 
                                     ORDER BY Count desc """)
state_counts.show(10)

In [ ]:
state_counts = spark.sql(r"""SELECT instructor_name, COUNT(instructor_name) AS Count 
                                     FROM data 
                                     GROUP BY instructor_name 
                                     ORDER BY Count desc """)
state_counts.show(10)

In [ ]:
state_counts = spark.sql(r"""SELECT category, COUNT(category) AS Count 
                                     FROM data 
                                     GROUP BY category 
                                     ORDER BY Count desc """)
state_counts.show(10)

In [ ]:
state_counts = spark.sql(r"""SELECT  subcategory,COUNT(subcategory) AS Count 
                                     FROM data 
                                     GROUP BY subcategory 
                                     ORDER BY Count desc """)
state_counts.show(10)

In [ ]:
state_counts = spark.sql(r"""SELECT is_paid, COUNT(is_paid) AS Count 
                                     FROM data 
                                     GROUP BY is_paid 
                                     ORDER BY Count desc """)
state_counts.show(2)

In [ ]:
state_counts = spark.sql(r"""SELECT topic, COUNT(topic) AS Count 
                                     FROM data 
                                     GROUP BY topic 
                                     ORDER BY Count desc """)
state_counts.show(10)

# Binary Featuers


In [ ]:
state_counts = spark.sql(r"""SELECT category,MAX(num_subscribers) AS Total_subscribers 
                                     FROM data 
                                     GROUP BY category 
                                     ORDER BY Total_subscribers desc """)
state_counts.show(13)

In [ ]:
state_counts = spark.sql(r"""SELECT title ,MAX(num_subscribers) AS Total_subscribers 
                                     FROM data 
                                     GROUP BY title 
                                     ORDER BY Total_subscribers desc """)
state_counts.show(10)

In [ ]:
state_counts = spark.sql(r"""SELECT category, avg(price) as avg
                                     FROM data
                                     GROUP BY category 
                                     ORDER BY avg desc """)
state_counts.show(10)

In [ ]:
state_counts = spark.sql(r"""SELECT category, subcategory, COUNT(subcategory) AS Count 
                                     FROM data
                                     GROUP BY subcategory,category
                                    ORDER BY category desc """)
state_counts.show(15)

In [ ]:
state_counts = spark.sql(r"""SELECT instructor_name ,SUM(title) AS Total_courses  
                                     FROM data 
                                     GROUP BY instructor_name 
                                     ORDER BY Total_courses desc """)
state_counts.show(10)

In [ ]:
state_counts = spark.sql(r"""SELECT  category, MAX(instructor_name) AS Total_courses  
                                     FROM data 
                                     GROUP BY category 
                                     ORDER BY Total_courses desc """)
state_counts.show(15)

# Complex Query 

In [ ]:
state_counts = spark.sql(r"""SELECT title,category,instructor_name, num_subscribers AS Total_subscribers 
                                     FROM data 
                                     ORDER BY Total_subscribers desc """)
state_counts.show(10)

In [ ]:
state_counts = spark.sql(r"""SELECT instructor_name, category, avg(price) as avg_price
                                     FROM data
                                     GROUP BY instructor_name,category 
                                      ORDER BY avg_price  """)
state_counts.show(15)